Summary of the Workflow:
- Fine-tune a pre-trained model in Colab using the Hugging Face transformers library.
- Push the fine-tuned model to Hugging Face Hub.
- Deploy the model on Hugging Face Spaces using Gradio.
- Share your Space for others to interact with your model.

In [1]:
pip install huggingface_hub

Step 1: Setup Environment in Google Colab
Open a Colab Notebook and make sure you’re connected to a GPU.

Install the Hugging Face transformers, datasets, and huggingface_hub libraries if you haven’t already:


Login to Hugging Face from Colab: You will need an authentication token from Hugging Face to push the model to the Hub. Go to Hugging Face to get your token.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Step 2: Load and Fine-tune a Pre-trained Model
Let’s use BERT for a text classification task. You can fine-tune on any dataset of your choice (e.g., imdb for sentiment analysis).

Load the dataset using datasets:

In [3]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from datasets import load_dataset

# Load a dataset, e.g., IMDb for sentiment analysis
dataset = load_dataset("imdb")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Preprocess the dataset by tokenizing the text:



In [5]:
from transformers import AutoTokenizer

# Load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=128)

# Tokenize the dataset
'''dataset.map is a method provided by the Hugging Face datasets library that allows you to apply a function to every element in a dataset. It is often used for data preprocessing, such as tokenization, data augmentation, or feature extraction. The method applies the function in a vectorized way to efficiently transform the dataset.'''
tokenized_datasets = dataset.map(preprocess_function, batched=True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Prepare the model for fine-tuning:



In [6]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load a pre-trained BERT model for sequence classification (binary classification)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./my-finetuned-bert2_next",          # output directory
    evaluation_strategy="epoch",     # evaluate during training
    save_strategy="epoch",           # save model every epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=128,   # batch size for training
    per_device_eval_batch_size=128,    # batch size for evaluation
    num_train_epochs=1,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
    #push_to_hub=True,                # Push to Hugging Face Hub
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Fine-tune the model
trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.290706


TrainOutput(global_step=196, training_loss=0.3676588486651985, metrics={'train_runtime': 996.3066, 'train_samples_per_second': 25.093, 'train_steps_per_second': 0.197, 'total_flos': 1644444096000000.0, 'train_loss': 0.3676588486651985, 'epoch': 1.0})

Step 3: Push Model to Hugging Face Hub
Once your model is fine-tuned, you can push it to the Hugging Face Hub. You need to have a repository on the Hugging Face Hub where your model will be pushed.

Push the model to the Hugging Face Hub:

In [7]:
trainer.push_to_hub("my-finetuned-bert2_next")
# Assuming 'trainer' is your training object and 'tokenizer' is your tokenizer object


events.out.tfevents.1730829955.dddccaede993.394.0:   0%|          | 0.00/5.57k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Emanai/my-finetuned-bert2_next/commit/d03ded22d01ddae74686f89b2747621889db58e2', commit_message='my-finetuned-bert2_next', commit_description='', oid='d03ded22d01ddae74686f89b2747621889db58e2', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
tokenizer.push_to_hub("my-finetuned-bert2_next", tokenizer=tokenizer)


README.md:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Emanai/my-finetuned-bert2_next/commit/d8df5b2a0af46ef70ce040fbd5a3ce12ddd8d58e', commit_message='Upload tokenizer', commit_description='', oid='d8df5b2a0af46ef70ce040fbd5a3ce12ddd8d58e', pr_url=None, pr_revision=None, pr_num=None)

# Use our model

In [9]:
from transformers import pipeline

classifier = pipeline("text-classification", model="Emanai/my-finetuned-bert2_next")
classifier("I am very sad sad")
'{}-{}'.format(classifier("I am very sad")[0]['score'], classifier("I am happy")[0]['label'])

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


'0.633283257484436-LABEL_1'

'0.5027313828468323-LABEL_1'

After this step, you can find your fine-tuned model at https://huggingface.co/your-username/my-finetuned-bert.


https://huggingface.co/Emanai/my-finetuned-bert.

# Deploy on Hugging Face Spaces

Step 4: Deploy on Hugging Face Spaces
Now, let's deploy the fine-tuned model on Spaces using a Gradio interface.

- Go to Hugging Face Spaces: https://huggingface.co/spaces and create a new Space.

- Choose Gradio as the framework for your Space.

- In the Space's repository, you'll need a app.py file. This file will define the Gradio interface for interacting with your model.

- Write app.py:

In [ ]:
'''
import gradio as gr
from transformers import pipeline

# Load the fine-tuned model from Hugging Face Hub
classifier = pipeline("text-classification", model="Emanai/my-finetuned-bert2")

def classify_text(text):
    return classifier(text)[0]

# Create a simple Gradio interface
iface = gr.Interface(fn=classify_text, inputs="text", outputs="label")

# Launch the Gradio app
iface.launch()
'''

requirements.txt


transformers
torch

- Test the interface: Once the Space is created and the app.py file is added, your Gradio app will launch, allowing you to interact with the fine-tuned model directly from the Hugging Face Space.